In [1]:
import os
import numpy as np 

from utils.read_data import read_data
from utils.re_referencing import rref_REST, rref_remove, rref_CSD, rref_average
from utils.data_filtering import filter_band_raw_to_raw
from utils.data_filtering import clean_windows_artifacts, reconstruct_signal

from metrics import analyse_network_by_metric

DEBUG:root:test


In [2]:
from typing import Optional, Callable

In [3]:
def create_tt(window_time: int, n_samples: int, i_window: int):
    return np.linspace(0, window_time, n_samples)+(i_window*window_time)

In [4]:
def preprocess_data(filepath: str, reconst_threshold: float, window_time: int, clean_threshold: float, rref_fn: Optional[Callable] = None):
    # Load data, filter on 0.1-100 [Hz] and rref REST
    raw = read_data(filepath)
    raw_filtered = filter_band_raw_to_raw(raw, [0.1, 100])
    
    # Reconstruct with raw_filtered (not re-ref)
    if reconst_threshold is not None:
        reconst_raw, ica, eliminated_comps = reconstruct_signal(raw_filtered, reconst_threshold, ch_names=["Fp1", "Fp2"]) #[ 0  1  5  6 13 16]
    else:
        reconst_raw = raw_filtered.copy()

    # Cleaning
    data_windows, rejected = clean_windows_artifacts(reconst_raw, 
                                                     window_time=window_time, 
                                                     std_threshold=clean_threshold, 
                                                     rref_fn=rref_fn
                                                    )
    return data_windows

In [5]:
# Params load data
data_folder = "../data/"

# Params Reconstruction
reconst_threshold = None #0.2

# Params Cleaning
clean_threshold = 0.6
window_time_cleaning = 10
rref_fn = rref_REST

# Params AIS
max_lag = 20
cmi_estimator = 'JidtGaussianCMI' #'JidtKraskovCMI' 'OpenCLKraskovCMI' 'JidtGaussianCMI'
local_values = False

settings = {
     'cmi_estimator': cmi_estimator, 
     'max_lag': max_lag,
     'local_values': local_values,
     #'n_perm_max_stat': 11,
     #'n_perm_min_stat': 11,
     #'alpha_max_stat': 0.1,
     #'n_perm_omnibus': 11,
     #'alpha_omnibus': 0.1,
     #'n_perm_max_seq': 11,
     #'alpha_max_seq': 0.1, 
     #'alpha_min_stat': 0.1,
     'verbose': False
}

save_folder = os.path.join(
    "../results/", 
    f"reconstth-{reconst_threshold}_cleanth-{clean_threshold}_windowtimecleaning-{window_time_cleaning}_{rref_fn.__name__}_maxlag-{max_lag}_cmiestimator-{cmi_estimator}".replace(".", "o")
)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [6]:
save_folder

'../results/reconstth-None_cleanth-0o6_windowtimecleaning-10_rref_REST_maxlag-20_cmiestimator-JidtGaussianCMI'

In [7]:
hc_filenames = list(filter(lambda x: x.startswith("h"), sorted(os.listdir(data_folder))))
sz_filenames = list(filter(lambda x: x.startswith("s"), sorted(os.listdir(data_folder))))

#filenames = hc_filenames[:3] + sz_filenames[:3]
filenames =  hc_filenames + sz_filenames
filenames

['h01.edf',
 'h02.edf',
 'h03.edf',
 'h04.edf',
 'h05.edf',
 'h06.edf',
 'h07.edf',
 'h08.edf',
 'h09.edf',
 'h10.edf',
 'h11.edf',
 'h12.edf',
 'h13.edf',
 'h14.edf',
 's01.edf',
 's02.edf',
 's03.edf',
 's04.edf',
 's05.edf',
 's06.edf',
 's07.edf',
 's08.edf',
 's09.edf',
 's10.edf',
 's11.edf',
 's12.edf',
 's13.edf',
 's14.edf']

In [ ]:
for filename in filenames:
    print("Preprocessing:", filename)
    filepath = os.path.join(data_folder, filename)
    data_windows = preprocess_data(filepath, 
                                   reconst_threshold=reconst_threshold,
                                   clean_threshold=clean_threshold,
                                   window_time=window_time_cleaning, 
                                   rref_fn=rref_fn
                                  )
    #all_data = np.concatenate(data_windows, axis=1)
    #all_data = all_data[:, :100] #Testing with first 100 samples
    
    #data_gpu = create_windows(all_data, time=100, )
    
    save_subject_folder = filename.split(".edf")[0]
    if not os.path.exists(os.path.join(save_folder, save_subject_folder)):
        os.makedirs(os.path.join(save_folder, save_subject_folder))
    
    for i_channel in [16, 17, 18]:
        ais_arr = []
        for i_window, data_window in enumerate(data_windows):
            print(f"Working on: {filename} in channel: {i_channel} with window: {i_window}")
            # Calculating AIS
            net_analysis = analyse_network_by_metric(data_window, channels=i_channel, metric="AIS", **settings)
            if local_values:
                ais = net_analysis.get_single_process(0)['ais'][0]
            else:
                ais = net_analysis.get_single_process(0)['ais']
            ais_arr.append(ais)
            # Store
            ais_save = np.array(ais_arr)
            save_filename = f"ais_ichannel-{i_channel}"
            save_filepath = os.path.join(save_folder, save_subject_folder, save_filename)
            np.savez(f"{save_filepath}.npz",
                    ais=ais_save)

Preprocessing: h01.edf
Extracting EDF parameters from /home/jarajaras/dynamicinfo-eeg/data/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 8251 samples (33.004 sec)



INFO:root:window number 32 rejected
INFO:root:window number 33 rejected
INFO:root:window number 36 rejected
INFO:root:window number 58 rejected
INFO:root:window number 60 rejected
INFO:root:window number 61 rejected
INFO:root:window number 65 rejected
INFO:root:window number 67 rejected
INFO:root:window number 70 rejected
INFO:root:window number 71 rejected
INFO:root:window number 72 rejected
INFO:root:window number 80 rejected
INFO:root:window number 81 rejected
INFO:root:window number 82 rejected
INFO:root:window number 83 rejected
INFO:root:window number 84 rejected
INFO:root:window number 86 rejected
INFO:root:window number 87 rejected


Working on: h01.edf in channel: 16 with window: 0
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 1
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 2
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 3
Adding data with pro


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 19
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 20
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 21
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

--------


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 38
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 39
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h01.edf in channel: 16 with window: 40
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

--------

...


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 21
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 22
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 23
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

--------


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 40
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 41
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 42
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

--------



---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 59
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 60
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 61
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

-------


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 78
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 79
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 16 with window: 80
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

--------


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 17 with window: 5
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 17 with window: 6
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

---------------------------- (3) final statistics
Working on: h04.edf in channel: 17 with window: 7
Adding data with properties: 1 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include candidates


---------------------------- (2) prune source candidates

-----------

In [ ]:
"""
i_channel = 16
# Calculating AIS
net_analysis = analyse_network_by_metric(all_data[:, :10000], channels=i_channel, metric="AIS", **settings)
ais = net_analysis.get_single_process(0)['ais'][0]
"""

In [ ]:
"""
for i_channel in [16]: #[16, 17, 18]
    # Calculating AIS
    n_samples = data_windows.shape[-1]
    tt = create_tt(window_time, n_samples, i_window)
    net_analysis = analyse_network_by_metric(data_windows[i_window], channels=i_channel, metric="AIS", **settings)
    ais = net_analysis.get_single_process(0)['ais'][0]
"""